# Production of duplicates, no duplicates, and salesforce reports
### Merging Inside Salespeople List With Revenue Report
### Dropping Duplicates by 'Opportunity'
### Renaming Columns
### Reformatting Market Names
### Writing 2 Versions to Separate CSVs

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import csv
pd.set_option('display.max_columns', None)

In [4]:
df1 = pd.DataFrame(pd.read_csv("report1619446221319.csv")) 
df1.reset_index(drop=True)
df2 = pd.read_csv('IHM_Inside Sales_New-Term-Transfer_08202020.csv')
df2.reset_index(drop=True)
# display(HTML(df2.to_html()))
# display(HTML(df1.to_html()))
print('The shape of df1 is: ' + str(df1.shape))

# 1. Remove 'Support - Remove' from main df
# 2. Rename Sales, 'Support - Keep'
print('unique sales types: '), print(df2['Type'].unique())
print('number of inside salespeople: '), print(len(df2['Employee Name']))

The shape of df1 is: (19434, 16)
unique sales types: 
['Support - Remove' 'Sales' 'Support - Keep']
number of inside salespeople: 
176


(None, None)

In [5]:
list = []

# this nested loop creates list of matches between the names in df2 (inside salespeople)
# and names of salespeople in df1 (revenue report)
def inside_sales_matches():
    for row in df1['Opportunity: Opportunity Owner: Full Name']:
        for name in df2['Employee Name']:
            if row == name:
                list.append(row)

                
inside_sales_matches()

# Casts the list made above to a dataframe so that it can be counted, and because I believe the code
# below will only take another dataframe for comparison
inside_sales_presence = pd.DataFrame(list, columns=['Employee Name'])

# list of unique matches between df1 and df2
inside_sales_presence_unique = np.unique(inside_sales_presence)

# produces list of matches sorted by frequency (size)
count = inside_sales_presence.groupby(['Employee Name']).size().sort_values(ascending=False)

# locates values where salesperson name in df1 matches salesperson name in df2 (unique matches list) and
# then changes the "home operating market" to "Inside Sales"
df1.loc[df1['Opportunity: Opportunity Owner: Full Name'].isin(inside_sales_presence_unique),
        'Opportunity: Opportunity Owner: Home Operating Market'] = 'Inside Sales'

df1 = df1.rename(columns = {'Opportunity: Opportunity ID': 'opportunity_id',
                            'Opportunity: Stage': 'stage',
                           'Opportunity: Opportunity Owner: Market Division': 'market_division',
                           'Opportunity: Opportunity Owner: Market Area': 'market_area',
                           'Opportunity: Opportunity Owner: Home Operating Market': 'market',
                           'Opportunity: Opportunity Owner: Full Name': 'ae_name',
                           'New Business': 'new_business',
                           'Opportunity: Account Name: Account Name': 'account',
                           'Opportunity: Opportunity Name': 'opportunity',
                           'Adwire Project ID': 'project_id',
                           'Form Date': 'form_date',
                           'Opportunity: Created Date': 'opportunity_created_date',
                           'Opportunity: Close Date': 'opportunity_closed_date',
                           'Opportunity: Gross Expected Revenue': 'gross_expected_revenue',
                           'Opportunity: Gross Amount': 'gross_amount',
                           'Form Created on Closed Opp': 'form_created_on_closed_opp',
                           'Opportunity: Opportunity Owner: Market Type': 'market_type'})

# Casts form_date to datetime format
# df1.form_date = df1.form_date.map(lambda x: pd.to_datetime(x).date())
# # strips '-' from values in form_date
# df1.form_date = df1.form_date.map(lambda x: x.strftime('%Y%m%d'))
df1.form_date = pd.to_datetime(df1['form_date'], errors='coerce').dt.strftime('%Y%m%d')
# df1.opportunity_closed_date = df1.opportunity_closed_date.map(lambda x: pd.to_datetime(x).date())
# df1.opportunity_closed_date = df1.opportunity_closed_date.map(lambda x: x.strftime('%Y%m%d'))

# assigns df1 to new variable; this is necessary, as opportunity ID column (which is dropped in the code below)
# must be present in the dataframe that is imported to salesforce_import_treatment file, but absent from the
# dataframes that are exported to "Data Studio Sources" (Google Sheets file)
df4 = df1

# Defines df3, the dataframe that will become the "duplicates removed" CSV for export to "Data Studio Sources"
df3 = df1.drop_duplicates(subset = ['opportunity_id'])

# Drops the Opportunity ID column from df1 and df3, as it is unnecessary for the data studio
df1 = df1.drop('opportunity_id', axis=1)
df3 = df3.drop('opportunity_id', axis=1)

# writes new dataframes to separate CSVs, resets index to col = 0
df1.to_csv('duplicates.csv', index = 0)
df3.to_csv('no_duplicates.csv', index = 0)
df4.to_csv('sf_duplicates.csv', index = 0)

# visual confirmation that no_duplicates is smaller, while duplicates and sf_duplicates are of the same row length
# but different column lengths
print('\'no_duplicates.csv\': ' + str(df3.shape), '\n',
      '\'duplicates.csv\': ' + str(df1.shape), '\n',
      '\'sf_duplicates.csv\': ' + str(df4.shape))

'no_duplicates.csv': (16476, 15) 
 'duplicates.csv': (19434, 15) 
 'sf_duplicates.csv': (19434, 16)


In [6]:
df1.loc[df1.form_date == df1.form_date.max()]

,stage,market_division,market_area,market,ae_name,OppOwnerEmail,opportunity,project_id,form_date,opportunity_created_date,opportunity_closed_date,gross_expected_revenue,gross_amount,form_created_on_closed_opp,market_type
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ac5iKAXAAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BBJv7EAgAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BBJluABwAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AqHEDCLQAAE,NaT,NaN,NaN,NaN,NaN,0,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AJjLSAMAAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AOwURAMwAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AQGELAagAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B8VMExEAAAA,NaT,NaN,NaN,NaN,NaN,0,NaN
271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B8VRugGAAAA,NaT,NaN,NaN,NaN,NaN,0,NaN


### List of Closed Deals From Promo

In [7]:
promo_rev = df3.loc[(df1.form_date >= '20201123') & 
                    (df3.form_date <= '20201124') & 
                    (df3.stage == 'Closed - Won') & 
                    (df3.form_created_on_closed_opp == '0') 
                    ]

export = pd.read_csv('Usage_Report_iheart_20201123-20201125.xlsm - Usage Report.csv')
db1 = pd.DataFrame(pd.read_csv('salesforce_data.csv'))

closed_deals = pd.merge(promo_rev, db1, on=['project_id', 'project_id'])
closed_deals.to_csv('closed_deals.csv')

/Users/arlisscoates/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [8]:
df3.loc[(df1.form_date >= '20201123') & 
                    (df3.form_date <= '20201124') & 
                    (df3.stage == 'Closed - Won') & 
                    (df3.form_created_on_closed_opp == '0') 
                    ]

,stage,market_division,market_area,market,ae_name,OppOwnerEmail,opportunity,project_id,form_date,opportunity_created_date,opportunity_closed_date,gross_expected_revenue,gross_amount,form_created_on_closed_opp,market_type


### Dataframe of No-Market AEs

In [9]:
# List of people without markets
no_market = df1[df1['market'].isnull()]
# display(HTML(df1.to_html()))


no_market_list = no_market.ae_name.to_list()
no_market_list = set(no_market_list)
no_market_df = pd.DataFrame(no_market_list, columns = ['name'])

no_market_df.to_csv('aes_no_market.csv', index=0)